Autor: Esteban Suárez Calvo

In [1]:
from datetime import timedelta
from time import time

from qiskit import QuantumCircuit
from qiskit.circuit.library import (
    RealAmplitudes,
    n_local,
    z_feature_map,
    zz_feature_map,
)
from qiskit_aer import AerSimulator
from qiskit_algorithms.optimizers import Optimizer
from qiskit_ibm_runtime import Sampler
from qiskit_machine_learning.algorithms import VQC
from qiskit_machine_learning.optimizers import COBYLA, CRS, SPSA
from qiskit_machine_learning.utils import algorithm_globals
from sklearn.datasets import load_breast_cancer
from sklearn.decomposition import PCA
from sklearn.metrics import recall_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.svm import SVC

# CCAM - Práctica 2: Redes Neuronales Cuánticas

Antes de empezar, fijaremos la semilla aleatoria, de manera que los resultados obtenidos en el notebook sean reproducibles.

In [2]:
algorithm_globals.random_seed = 1

## Preprocesado

### Normalización del dataset

Normalizamos el dataset utilizando la normalización MinMax.

In [3]:
data = load_breast_cancer()
X = data.data
y = data.target
X = MinMaxScaler().fit_transform(X)
print(f"Número de características = {X.shape[1]}")

Número de características = 30


### Reducción de dimensionalidad

Dado que la simulación de circuitos cuánticos escala exponencialmente con el número de qubits, trabajar con 30 características iniciales resulta inviable.

Por lo tango, aplicamos la técnica ***Principal Component Analysis (PCA)*** para  reducir la dimensionalidad a **12 características**.

Aunque esto implica una pérdida de información, es un compromiso necesario para ajustar el problema procesable por el simulador en un tiempo razonable.

Como veremos en los siguientes apartados, dependiendo del optimizador escogido, el tiempo de entrenamiento puede ser bastante elevado (horas), mientras que en otros dura cuestión de minutos. Por lo tanto, aunque resulta posible utilizar un mayor número de características, hay ciertos optimizadores para los cuales no podríamos haber realizado la simulación.

Por lo tanto, al elegir 12 características, estamos buscando un equilibrio entre reducir la dimensionalidad para poder utilizar varios optimizadores diferentes, pero sin eliminar demasiada información.

In [4]:
X = PCA(n_components=12).fit_transform(X)
number_of_features = X.shape[1]
print(f"Nuevo número de características = {number_of_features}")

Nuevo número de características = 12


### Dividir el conjunto entre entrenamiento y test

In [5]:
X_train, X_test, Y_train, Y_test = train_test_split(
    X, y, test_size=0.20, random_state=algorithm_globals.random_seed
)
print(f"Datos de entrenamiento: {len(X_train)}")
print(f"Datos de test: {len(X_test)}")

Datos de entrenamiento: 455
Datos de test: 114


### Clasificación con un SVM

Como ya hemos comentado, reducir la dimensionalidad de nuestro dataset conlleva cierta pérdida de información. Si esta pérdida es muy elevada, puede hacer que las clases que queremos clasificar ya no sean distinguibles entre sí y que, por tanto, ningún clasificador (ya sea clásico o cuántico), podrá resolver el problema.

Por este motivo, a continuación realizaremos una clasificación utilizando **máquinas de soporte vectorial (SVM)**, con el fin de comprobar que esto no sucede.

In [15]:
svc = SVC()

start = time()
svc.fit(X_train, Y_train)
total_time = time() - start
print(f"SVC entrenado en {total_time * 1000:.2} ms")

train_score = svc.score(X_train, Y_train)
test_score = svc.score(X_test, Y_test)
y_pred_svc = svc.predict(X_test)
svc_recall = recall_score(Y_test, y_pred_svc, pos_label=0)

print(f"SVC en entrenamiento: {train_score:.5}")
print(f"SVC en test (Accuracy): {test_score:.5}")
print(f"SVC en test (Recall): {svc_recall:.5}")

SVC entrenado en 6.5 ms
SVC en entrenamiento: 0.98242
SVC en test: 0.97368
SVC en entrenamiento: 0.98242
SVC en test (Accuracy): 0.97368
SVC en test (Recall): 0.92857


Como podemos observar, en cuestión de milisegundos, podemos obtener un `SVC` capaz de resolver el problema prácticamente en la totalidad de los casos.

## Desarrolla una QNN para predecir la clase de cada caso

A continuación, definimos una función que, dado un `feature_map`, un `ansatz` y un `optimizer` arbitrarios, crea el `VQC` correspondiente y lo entrena. Además, también imprime por pantalla el tiempo de entrenamiento.

Definimos también un callback para hacer más interactivo el proceso de entrenamiento, de manera que, a cada iteración, se muestre el número de la iteración y el *loss*:

In [7]:
def train_VQC(
    feature_map: QuantumCircuit, ansatz: QuantumCircuit, optimizer: Optimizer
) -> VQC:
    global X_train, Y_train

    vqc = VQC(
        feature_map=feature_map,
        ansatz=ansatz,
        optimizer=optimizer,
        callback=LossLogger().log_loss,
        sampler=Sampler(AerSimulator()),
    )

    start = time()
    vqc.fit(X_train, Y_train)
    total_time = time() - start
    print(f"Tiempo total de entrenamiento: {timedelta(seconds=total_time)}")

    return vqc


class LossLogger:
    def __init__(self):
        self._iteration = 0

    def log_loss(self, weights, obj_func_eval):
        self._iteration += 1
        print(f"Iteración {self._iteration} - Loss: {obj_func_eval:.4f}")

Definimos también una función para poder evaluar el clasificador.

Debido a que detectar un tumor **maligno** es más importante que detectar un tumor **benigno**, además de calcular el *accuracy*, también calcularemos el ***recall***, puesto que esta métrica mide la tasa de verdaderos positivos:

$$ 
\text{Recall} = \frac{\text{Verdaderos Positivos}}{\text{Verdaderos Positivos} + \text{Falsos Negativos}} 
$$

In [8]:
def evaluate_VQC(vqc: VQC) -> None:
    global X_train, Y_train

    # Accuracy
    train_score = vqc.score(X_train, Y_train)
    print(f"SVC en entrenamiento: {train_score:.5}")
    test_score = vqc.score(X_test, Y_test)
    print(f"SVC en test: {test_score:.5}")

    # Recall
    y_pred = vqc.predict(X_test)
    recall = recall_score(Y_test, y_pred, pos_label=0)  # 0 corresponde a maligno
    print(f"Recall (Sensibilidad clase Maligna): {recall:.5}")

## Ejecución con distintos `VQC`

A continuación realizamos una serie de pruebas, con el objetivo de comparar diferentes `VQC`:

- **VQC 1**

In [9]:
feature_map = zz_feature_map(number_of_features)
ansatz = n_local(number_of_features, "ry", "cx", "linear")
optimizer = COBYLA(maxiter=20)

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Iteración 1 - Loss: 0.9990
Iteración 2 - Loss: 0.9994
Iteración 3 - Loss: 1.0009
Iteración 4 - Loss: 0.9881
Iteración 5 - Loss: 0.9903
Iteración 6 - Loss: 0.9873
Iteración 7 - Loss: 0.9915
Iteración 8 - Loss: 0.9905
Iteración 9 - Loss: 0.9918
Iteración 10 - Loss: 0.9955
Iteración 11 - Loss: 0.9893
Iteración 12 - Loss: 0.9882
Iteración 13 - Loss: 0.9898
Iteración 14 - Loss: 0.9879
Iteración 15 - Loss: 1.0061
Iteración 16 - Loss: 0.9932
Iteración 17 - Loss: 0.9900
Iteración 18 - Loss: 0.9902
Iteración 19 - Loss: 0.9893
Iteración 20 - Loss: 0.9907
Tiempo total de entrenamiento: 0:04:33.707907
SVC en entrenamiento: 0.52088
SVC en test: 0.60526
Recall (Sensibilidad clase Maligna): 0.45238


- **VQC 2**

In [10]:
feature_map = z_feature_map(number_of_features)
ansatz = n_local(number_of_features, "ry", "cx", "linear")
optimizer = COBYLA(maxiter=20)

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Iteración 1 - Loss: 0.9907
Iteración 2 - Loss: 0.8769
Iteración 3 - Loss: 0.9359
Iteración 4 - Loss: 0.9094
Iteración 5 - Loss: 0.8770
Iteración 6 - Loss: 0.8743
Iteración 7 - Loss: 0.8750
Iteración 8 - Loss: 0.8752
Iteración 9 - Loss: 0.8760
Iteración 10 - Loss: 0.8777
Iteración 11 - Loss: 0.8744
Iteración 12 - Loss: 0.8728
Iteración 13 - Loss: 0.8748
Iteración 14 - Loss: 0.8936
Iteración 15 - Loss: 0.8685
Iteración 16 - Loss: 0.9180
Iteración 17 - Loss: 0.8640
Iteración 18 - Loss: 0.8674
Iteración 19 - Loss: 0.8675
Iteración 20 - Loss: 0.8674
Tiempo total de entrenamiento: 0:01:47.919220
SVC en entrenamiento: 0.6967
SVC en test: 0.67544
Recall (Sensibilidad clase Maligna): 0.2381


- **VQC 3**

In [11]:
feature_map = zz_feature_map(number_of_features)
ansatz = RealAmplitudes(number_of_features).decompose()
optimizer = COBYLA(maxiter=20)

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Iteración 1 - Loss: 0.9965
Iteración 2 - Loss: 0.9892
Iteración 3 - Loss: 0.9938
Iteración 4 - Loss: 0.9925
Iteración 5 - Loss: 0.9964
Iteración 6 - Loss: 0.9965
Iteración 7 - Loss: 0.9917
Iteración 8 - Loss: 0.9906
Iteración 9 - Loss: 0.9881
Iteración 10 - Loss: 0.9923
Iteración 11 - Loss: 0.9901
Iteración 12 - Loss: 0.9962
Iteración 13 - Loss: 0.9941
Iteración 14 - Loss: 0.9984
Iteración 15 - Loss: 0.9880
Iteración 16 - Loss: 0.9906
Iteración 17 - Loss: 0.9864
Iteración 18 - Loss: 0.9849
Iteración 19 - Loss: 0.9859
Iteración 20 - Loss: 0.9919
Tiempo total de entrenamiento: 0:04:33.217853
SVC en entrenamiento: 0.61099
SVC en test: 0.55263
Recall (Sensibilidad clase Maligna): 0.5


- **VQC 4**

In [12]:
feature_map = zz_feature_map(number_of_features)
ansatz = n_local(number_of_features, "ry", "cx", "linear", reps=3)
optimizer = COBYLA(maxiter=20)

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Iteración 1 - Loss: 0.9908
Iteración 2 - Loss: 0.9989
Iteración 3 - Loss: 0.9972
Iteración 4 - Loss: 0.9839
Iteración 5 - Loss: 0.9895
Iteración 6 - Loss: 0.9884
Iteración 7 - Loss: 0.9912
Iteración 8 - Loss: 0.9874
Iteración 9 - Loss: 0.9877
Iteración 10 - Loss: 0.9894
Iteración 11 - Loss: 0.9879
Iteración 12 - Loss: 0.9894
Iteración 13 - Loss: 0.9946
Iteración 14 - Loss: 0.9852
Iteración 15 - Loss: 0.9953
Iteración 16 - Loss: 0.9909
Iteración 17 - Loss: 0.9880
Iteración 18 - Loss: 0.9870
Iteración 19 - Loss: 0.9921
Iteración 20 - Loss: 0.9879
Tiempo total de entrenamiento: 0:04:33.014091
SVC en entrenamiento: 0.6022
SVC en test: 0.52632
Recall (Sensibilidad clase Maligna): 0.47619


- **VQC 5**

In [13]:
feature_map = z_feature_map(number_of_features)
ansatz = n_local(number_of_features, "ry", "cx", "linear", reps=3)
optimizer = COBYLA(maxiter=20)

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Iteración 1 - Loss: 1.3347
Iteración 2 - Loss: 1.1137
Iteración 3 - Loss: 1.0737
Iteración 4 - Loss: 1.0748
Iteración 5 - Loss: 1.0727
Iteración 6 - Loss: 1.0736
Iteración 7 - Loss: 1.0775
Iteración 8 - Loss: 1.0725
Iteración 9 - Loss: 1.0742
Iteración 10 - Loss: 1.0756
Iteración 11 - Loss: 1.0711
Iteración 12 - Loss: 1.0771
Iteración 13 - Loss: 1.0756
Iteración 14 - Loss: 0.8937
Iteración 15 - Loss: 0.8855
Iteración 16 - Loss: 0.8803
Iteración 17 - Loss: 0.8886
Iteración 18 - Loss: 0.8827
Iteración 19 - Loss: 0.8813
Iteración 20 - Loss: 0.8831
Tiempo total de entrenamiento: 0:01:49.113893
SVC en entrenamiento: 0.75385
SVC en test: 0.70175
Recall (Sensibilidad clase Maligna): 0.35714


- **VQC 6**

In [14]:
feature_map = zz_feature_map(number_of_features)
ansatz = n_local(number_of_features, "ry", "cx", "linear")
optimizer = SPSA(maxiter=20)

vqc = train_VQC(feature_map, ansatz, optimizer)
evaluate_VQC(vqc)

No gradient function provided, creating a gradient function. If your Sampler requires transpilation, please provide a pass manager.


Tiempo total de entrenamiento: 0:20:45.689359
SVC en entrenamiento: 0.63516
SVC en test: 0.62281
Recall (Sensibilidad clase Maligna): 0.28571


## Resultados

Tenemos entonces los siguientes resultados:

| Clasificador | `feature_map`    | `ansatz`          | `optimizer` | Accuracy en entrenamiento | Accuracy en test | Recall en test | Tiempo de entrenamiento |     |
| ------------ | ---------------- | ----------------- | ----------- | ------------------------- | ---------------- | -------------- | ----------------------- | --- |
| **VQC 1**    | `zz_feature_map` | `n_local`         | `COBYLA`    | 0.52088                   | 0.60526          | 0.45238        | 0:04:33                 |     |
| **VQC 2**    | `z_feature_map`  | `n_local`         | `COBYLA`    | 0.6967                    | 0.67544          | 0.2381         | 0:01:47                 |     |
| **VQC 3**    | `zz_feature_map` | `RealAmplitudes`  | `COBYLA`    | 0.61099                   | 0.55263          | 0.5            | 0:04:33                 |     |
| **VQC 4**    | `zz_feature_map` | `n_local(reps=3)` | `COBYLA`    | 0.6022                    | 0.52632          | 0.47619        | 0:04:33                 |     |
| **VQC 5**    | `z_feature_map`  | `n_local(reps=3)` | `COBYLA`    | 0.75385                   | 0.70175          | 0.35714        | 0:01:49                 |     |
| **VQC 6**    | `zz_feature_map` | `n_local`         | `SPSA`      | 0.63516                   | 0.62281          | 0.28571        | 0:20:45                 |     |


## Conclusiones

Los resultados evidencian una brecha significativa entre los paradigmas clásico y cuántico para este problema específico. Mientras que un SVC clásico alcanza un accuracy del 100% en unos pocos milisegundos, los VQC presentan serias dificultades de aprendizaje, independientemente de los `feature_map`, `ansatz` y `optimizer` escogidos.

Aunque en uno de los VQC (VQC 5) obtenemos un accuracy del 70.1% para test, este resultado puede ser bastante engañoso. Su recall para la clase maligna es extremadamente bajo (35.7%), lo que indica que el modelo está sesgado hacia la clase mayoritaria (begigna) y falla en detectar la gran mayoría de casos positivos. Este sesgo se presenta también en el resto de modelos, pues el máximo recall que alcanzamos es en VQC 3 (50%). En un contexto de diagnóstico médico, donde la prioridad es minimizar los falsos negativos, un recall inferior a un 90% resulta inviable, independientemente de su accuracy global.

Respecto al entrenamiento, se observa que la función de loss se mantiene prácticamente constante en todos los epochs, sin lograr converger. Este comportamiento se debe a un fenómeno conocido como ***Barren Plateaus***. Este problema consiste en que el gradiente de la función de coste tiende a desvanecerse. Es decir, el optimizador navega por una superficie plana donde no encuentra dirección de descenso, lo que impide el aprendizaje.

Finalmente, el éxito del SVC con solo 12 características nos confirma que los datos son separables en el espacio clásico reducido. La gran fortaleza de los VQC es proyectar datos complejos a un espacio de Hilbert de alta dimensionalidad para facilitar su separación, pero en este caso esa proyección resulta redundante. Por lo tanto, podemos concluir que, cuando los datos no tienen una complejidad estructural alta, usar VQC supone una sobrecarga computacional inmensa que no aporta una ventaja real frente a métodos clásicos robustos.